<a href="https://colab.research.google.com/github/wisdomscode/AI-Lab-Deep-Learning-PyTorch/blob/main/AI_Lab_Project_6_Social_Media_Marketing_at_WQU_6_6_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Getting Ready

We'll be working with Stable Diffusion, so we'll need to import `diffusers` and `torch`.

**Task 6.6.1:** Import `torch` and the `diffusers` modules.

In [ ]:
import diffusers
import torch

**Task 6.6.2:** Set the `device` and `dtype` as we did in the lessons.

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
    dtype = torch.float16
else:
    device = "cpu"
    dtype = torch.float32

### Stable Diffusion

We'll need a Stable Diffusion pipeline to be able to produce images. As we did in the lesson, we'll create a function that returns one.

**Task 6.6.3:** Create a function `load_model` that takes no arguments. It should load the Stable Diffusion v1.4 model from Hugging Face and place it on the correct device.  It should return the Stable Diffusion pipeline that gets created.

In [ ]:
def load_model():
    MODEL_NAME = "CompVis/stable-diffusion-v1-4"
    pipeline = diffusers.AutoPipelineForText2Image.from_pretrained(
        MODEL_NAME, torch_dtype=dtype
    )
    return pipeline

In [ ]:
pipeline = load_model()

print(f"On device {pipeline.device}")
pipeline


#output
Loading pipeline components...: 100%
 7/7 [00:04<00:00,  1.27it/s]
On device cpu
StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.32.2",
  "_name_or_path": "CompVis/stable-diffusion-v1-4",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

We'll want a function to generate images, as we had before. We'll start with the same one, but we'll expand it as we continue along.

**Task 6.6.4:** Create a function `generate_images(prompt, pipeline, n)`.  It should take three arguments:
- `prompt`: An image generation prompt, as a string.
- `pipeline`: A Stable Diffusion pipeline object.
- `n`: The number of images to create, as an integer.

It should return a list of PIL Images of length `n`.

In [ ]:
def generate_images(prompt, pipeline, n):
    return pipeline([prompt] * n).images

In [ ]:
images = generate_images("Tree with autumn foliage", pipeline, 2)

images[0]

This is a nice starting place. But the `pipeline` takes more arguments, to give users more flexibility with how images are generated. Let's add those in. We'll start with the guidance scale, which we can change with the `guidance_scale` argument in the pipeline.

**Task 6.6.5:** Change your `generate_images` to also take a `guidance` argument, with a default value of `7.5`. Also change your call to pipeline to include the argument `guidance_scale=guidance`.

In [ ]:
def generate_images(prompt, pipeline, n, guidance=7.5):
    output = pipeline(
        [prompt] * n, guidance_scale=guidance
    )
    return output.images

In [ ]:
images = generate_images("Tree with autumn foliage", pipeline, 2, guidance=3.0)

images[0]

We can also change how many steps the model takes, by adjusting the `num_inference_steps` argument to the pipeline. Let's add that as well.

**Task 6.6.6:** Change your `generate_images` to also take a `steps` argument, with a default value of `50`. Also change your call to pipeline to include the argument `num_inference_steps=steps`.

In [ ]:
def generate_images(prompt, pipeline, n, guidance=7.5, steps=50):
    output = pipeline(
        [prompt] * n, guidance_scale=guidance, num_inference_steps=steps
    )
    return output.images

In [ ]:
images = generate_images(
    "Tree with autumn foliage", pipeline, 2, guidance=7, steps=20
)

images[0]

There are other arguments to the pipeline we could change, but this should be enough for now. We don't want to overwhelm our users (or ourselves)!

One thing we saw was that we could add styles. This was actually a somewhat annoying process, so it would be nice for our users if we could do this for them. Here is a dictionary that presents different Stable Diffusion styles along with short names as keys. It'll be handy for the Streamlit app:

In [ ]:
style_dict = {
    'none': '',
    'anime': 'cartoon, animated, Studio Ghibli style, cute, Japanese animation',
    'photo': 'photograph, film, 35 mm camera',
    'video game': 'rendered in unreal engine, hyper-realistic, volumetric lighting, --ar 9:16 --hd --q 2',
    'watercolor': 'painting, watercolors, pastel, composition'
}


Now we should adjust our `generate_images` to take the style name, and add the style string to the prompt. Later, our user will be able to select the nice short name and our app will take care of the rest.

**Task 6.6.7:** Change your `generate_images` to also take a `style` argument, with a default value of `"none"`. In the function, look up the style string based on the `style` argument, and add that string to the prompt

In [ ]:
def generate_images(prompt, pipeline, n, guidance=7.5, steps=50, style="none"):
    style_text = style_dict[style]
    output = pipeline(
        [prompt + style_text] * n, guidance_scale=guidance, num_inference_steps=steps
    )
    return output.images

In [ ]:
images = generate_images(
    "Tree with autumn foliage", pipeline, 2, steps=20, style="anime"
)

images[0]

### Streamlit App

Congratulations! You have successfully customized Stable Diffusion. Now it's time to create a Streamlit app that will present this to any user.

> **The rest of this lesson is entirely optional and it's not graded.**

Open the app script located in `app.py` and fill in the code that you have created during this lesson: the imports, the `device` and `dtype` and the functions.

You'll find that we have already created a `main()` function that will use your function's parameters.

Once you're ready to try it out, run the following in a terminal:

```bash
streamlit run app.py --browser.serverAddress 0.0.0.0 --server.port 9000
```

Now use the tab switcher to go to the `Streamlit App` tab.

In [ ]:
# in app.py

# import needed libraries
import diffusers
import torch


# Set the device and `dtype` for GPUs

# The dictionary mapping style names to style strings
style_dict = {
    "none": "",
    "anime": "cartoon, animated, Studio Ghibli style, cute, Japanese animation",
    # A photograph on film suggests an artistic approach
    "photo": "photograph, film, 35 mm camera",
    "video game": "rendered in unreal engine, hyper-realistic, volumetric lighting, --ar 9:16 --hd --q 2",
    "watercolor": "painting, watercolors, pastel, composition",
}

# Load Stable Diffusion (load_model function)
def load_model():
    MODEL_NAME = "CompVis/stable-diffusion-v1-4"
    pipeline = diffusers.AutoPipelineForText2Image.from_pretrained(
        MODEL_NAME
    )
    return pipeline


# The generate_images function
def generate_images(prompt, pipeline, n, guidance=7.5, steps=50, style="none"):
    style_text = style_dict[style]
    output = pipeline(
        [prompt + style_text] * n, guidance_scale=guidance, num_inference_steps=steps
    )
    return output.images


# The main function
def main():
    st.title("Stable Diffusion GUI")

    num_images = st.sidebar.number_input("Number of Images", min_value=1, max_value=10)
    prompt = st.sidebar.text_area("Text-to-Image Prompt")

    guidance_help = "Lower values follow the prompt less strictly. Higher values risk distored images."
    guidance = st.sidebar.slider("Guidance", 2.0, 15.0, 7.5, help=guidance_help)

    steps_help = "More steps produces better images but takes longer."
    steps = st.sidebar.slider("Steps", 10, 150, 50, help=steps_help)

    style = st.sidebar.selectbox("Style", options=style_dict.keys())

    generate = st.sidebar.button("Generate Images")
    if generate:
        with st.spinner("Generating images..."):
            pipeline = load_model()
            images = generate_images(
                prompt, pipeline, num_images, guidance, steps, style
            )
            for im in images:
                st.image(im)


if __name__ == "__main__":
    main()
